In [7]:
# 1. Install Required Libraries
!pip install pandas numpy matplotlib seaborn scikit-learn ipywidgets

# 2. Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import ipywidgets as widgets
from IPython.display import display

# 3. Upload the Dataset
from google.colab import files
uploaded = files.upload()

# 4. Load and Preprocess Data
filename = 'covtype.csv'  # Change if different
df = pd.read_csv(filename)
print("✅ Dataset Loaded Successfully")
print("\nShape:", df.shape)
df.head()

# 5. Feature/Target Split + Scaling
X = df.drop('Cover_Type', axis=1)
y = df['Cover_Type']

# Reduce data size for performance (optional)
X_sample, _, y_sample, _ = train_test_split(X, y, train_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_sample)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_sample, test_size=0.3, random_state=42)
print("✅ Data Splitting Done")

# 6. Train & Evaluate ML Models
models = {
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier()
}

results = {}

for name, model in models.items():
    print(f"\n🔹 {name} 🔹")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))
    results[name] = acc

# 7. KMeans Clustering with PCA
kmeans = KMeans(n_clusters=7, random_state=42)  # Cover_Type has 7 classes
labels = kmeans.fit_predict(X_test)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_test)

plt.figure(figsize=(10,6))
plt.scatter(X_pca[:,0], X_pca[:,1], c=labels, cmap='tab10', s=10)
plt.title('K-Means Clustering (PCA Reduced)')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()

# 8. Accuracy Comparison
print("\n✅ Final Model Comparison:")
for model_name, accuracy in results.items():
    print(f"{model_name}: {accuracy:.4f}")

# 9. Interactive Prediction (First 10 features)
feature_names = list(X.columns[:10])
example_values = [0.0] * 10

print("\n📥 Enter feature values to predict Cover_Type (first 10 features):")

feature_widgets = []
input_boxes = []
for i, feature in enumerate(feature_names):
    label = widgets.Label(value=feature, layout=widgets.Layout(width='220px'))
    input_box = widgets.FloatText(value=example_values[i], layout=widgets.Layout(width='150px'))
    feature_widgets.append(widgets.HBox([label, input_box]))
    input_boxes.append(input_box)

display(widgets.VBox(feature_widgets))

predict_button = widgets.Button(description="Predict Cover_Type", button_style='success')
output = widgets.Output()

def on_predict_clicked(b):
    global models, X, scaler
    with output:
        output.clear_output()

        input_values = [box.value for box in input_boxes]

        if all(v == 0 for v in input_values):
            print("⚠️ Please enter valid feature values. All zeros are not allowed.")
            return

        full_input = np.zeros(X.shape[1])
        full_input[:10] = input_values

        input_df = pd.DataFrame([full_input], columns=X.columns)
        scaled_input = scaler.transform(input_df)

        rf_model = models["Random Forest"]
        prediction_proba = rf_model.predict_proba(scaled_input)[0]

        sorted_indices = np.argsort(prediction_proba)[::-1]
        print("🔍 Top 3 Predicted Cover Types:")
        for rank in range(3):
            class_idx = sorted_indices[rank]
            proba = prediction_proba[class_idx] * 100
            print(f"{rank+1}. Cover_Type {class_idx+1} ({proba:.2f}%)")

        best_class = sorted_indices[0] + 1
        print(f"\n✅ Final Predicted Cover_Type: {best_class}")

predict_button.on_click(on_predict_clicked)
display(predict_button, output)


Saving covtype.csv to covtype (4).csv
✅ Dataset Loaded Successfully

Shape: (581012, 55)
✅ Data Splitting Done

🔹 K-Nearest Neighbors 🔹
Accuracy: 0.8649
              precision    recall  f1-score   support

           1       0.88      0.86      0.87     12840
           2       0.88      0.90      0.89     16843
           3       0.82      0.83      0.83      2152
           4       0.75      0.65      0.70       175
           5       0.71      0.57      0.63       554
           6       0.70      0.61      0.65      1052
           7       0.88      0.85      0.87      1245

    accuracy                           0.86     34861
   macro avg       0.80      0.75      0.78     34861
weighted avg       0.86      0.86      0.86     34861


🔹 Naive Bayes 🔹
Accuracy: 0.0897
              precision    recall  f1-score   support

           1       0.15      0.02      0.04     12840
           2       0.91      0.01      0.02     16843
           3       0.30      0.40      0.34      2152


✅ Final Model Comparison:
K-Nearest Neighbors: 0.8649
Naive Bayes: 0.0897
Random Forest: 0.9057
Decision Tree: 0.8645

📥 Enter feature values to predict Cover_Type (first 10 features):


Button(button_style='success', description='Predict Cover_Type', style=ButtonStyle())

Output()